In [1]:
!pip install transformers datasets torch


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


C:\Users\chila\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
pip install tensorflow

   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
    --------------------------------------- 4.2/332.0 MB 28.8 MB/s eta 0:00:12
   - -------------------------------------- 12.3/332.0 MB 34.4 MB/s eta 0:00:10
   -- ------------------------------------- 21.2/332.0 MB 37.2 MB/s eta 0:00:09
   --- ------------------------------------ 28.6/332.0 MB 36.4 MB/s eta 0:00:09
   ---- ----------------------------------- 36.7/332.0 MB 37.2 MB/s eta 0:00:08
   ----- ---------------------------------- 46.4/332.0 MB 38.8 MB/s eta 0:00:08
   ------ --------------------------------- 54.8/332.0 MB 39.2 MB/s eta 0:00:08
   ------- -------------------------------- 63.2/332.0 MB 39.6 MB/s eta 0:00:07
   -------- ------------------------------- 72.6/332.0 MB 40.4 MB/s eta 0:00:07
   --------- ------------------------------ 80.5/332.0 MB 39.9 MB/s eta 0:00:07
   ---------- ----------------------------- 89.7/332.0 MB 40.3 MB/s eta 0:00:07
   ------------ --------------------------- 99.9/3


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import tensorflow as tf
print[tf.test.is_gpu_available()]

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


TypeError: 'builtin_function_or_method' object is not subscriptable

In [8]:
# Load dataset (2% of the training set)
dataset = load_dataset('amazon_polarity', split='train[:1%]')

# Split dataset into train and test (e.g., 90% train, 10% test)
split = dataset.train_test_split(test_size=0.1)
train_dataset = split['train']
test_dataset = split['test']


In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['content'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=32)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=32)

# Set formats for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map: 100%|████████████████████████████████████████████████████████████████| 3600/3600 [00:02<00:00, 1554.63 examples/s]


In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [8]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to=[]  # Disable all integrations including wandb
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


C:\Users\chila\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", eval_results)


In [ ]:
model.save_pretrained('./fine_tuned_bert_amazon')
tokenizer.save_pretrained('./fine_tuned_bert_amazon')
